In [1]:
import sys
print('Python Version %s' % sys.version)

Python Version 3.6.5 |Anaconda, Inc.| (default, Apr 29 2018, 16:14:56) 
[GCC 7.2.0]


In [2]:
!pip list

Package                            Version   
---------------------------------- ----------
absl-py                            0.9.0     
alabaster                          0.7.10    
anaconda-client                    1.6.14    
anaconda-project                   0.8.2     
asn1crypto                         0.24.0    
astor                              0.8.1     
astroid                            1.6.3     
astropy                            3.0.2     
attrs                              18.1.0    
Automat                            0.3.0     
autovizwidget                      0.15.0    
awscli                             1.18.39   
Babel                              2.5.3     
backcall                           0.1.0     
backports.shutil-get-terminal-size 1.0.0     
bcrypt                             3.1.7     
beautifulsoup4                     4.6.0     
bitarray                           0.8.1     
bkcharts                           0.2       
blaze                             

In [3]:
%%bash
conda list

# packages in environment at /home/ec2-user/anaconda3/envs/python3:
#
# Name                    Version                   Build  Channel
_libgcc_mutex             0.1                        main  
absl-py                   0.9.0                     <pip>
alabaster                 0.7.10           py36h306e16b_0  
anaconda-client           1.6.14                   py36_0  
anaconda-project          0.8.2            py36h44fb852_0  
argparse                  1.4.0                     <pip>
asn1crypto                0.24.0                   py36_0  
astor                     0.8.1                     <pip>
astroid                   1.6.3                    py36_0  
astropy                   3.0.2            py36h3010b51_1  
attrs                     18.1.0                   py36_0  
Automat                   0.3.0                     <pip>
autovizwidget             0.15.0                    <pip>
awscli                    1.18.39                   <pip>
babel                     2.5.3    

In [4]:
%%html

<iframe src="https://datascienceonaws.com" width="800px" height="600px"/>

# 1. Install the CLI tools

_Note:  Make sure the SageMaker role has access to create EKS Cluster._

In [5]:
%%bash

source ~/.bash_profile

#### Install `eksctl`
# To get started we'll first install the `awscli` and `eksctl` CLI tools. [eksctl](https://eksctl.io) simplifies the process of creating EKS clusters.

pip install awscli --upgrade --user

curl --silent --location "https://github.com/weaveworks/eksctl/releases/latest/download/eksctl_$(uname -s)_amd64.tar.gz" | tar xz -C /tmp

sudo mv /tmp/eksctl /usr/local/bin

eksctl version

#### Install `kubectl`
# `kubectl` is a command line interface for running commands against Kubernetes clusters. 
# Run the following to install Kubectl

curl --location -o ./kubectl https://amazon-eks.s3.us-west-2.amazonaws.com/1.15.10/2020-02-22/bin/linux/amd64/kubectl

chmod +x ./kubectl

sudo mv ./kubectl /usr/local/bin

kubectl version --short --client

#### Install `aws-iam-authenticator`

curl -o aws-iam-authenticator https://amazon-eks.s3.us-west-2.amazonaws.com/1.15.10/2020-02-22/bin/linux/amd64/aws-iam-authenticator

chmod +x ./aws-iam-authenticator

sudo mv aws-iam-authenticator /usr/local/bin

aws-iam-authenticator version

#### Install jq and envsubst (from GNU gettext utilities) 
sudo yum -y install jq gettext

#### Verify the binaries are in the path and executable
for command in kubectl jq envsubst
  do
    which $command &>/dev/null && echo "$command in path" || echo "$command NOT FOUND"
  done

echo "Completed"

0.17.0
Client Version: v1.15.10-eks-bac369
{"Version":"v0.5.0","Commit":"1cfe2a90f68381eacd7b6dcfa2bf689e76eb8b4b"}
Loaded plugins: dkms-build-requires, priorities, update-motd, upgrade-helper,
              : versionlock
Package jq-1.5-1.2.amzn1.x86_64 already installed and latest version
Package gettext-0.18.1.1-9.1.11.amzn1.x86_64 already installed and latest version
Nothing to do
kubectl in path
jq in path
envsubst in path
Completed


You should consider upgrading via the 'pip install --upgrade pip' command.
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 55.1M  100 55.1M    0     0  90.4M      0 --:--:-- --:--:-- --:--:-- 90.4M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 33.6M  100 33.6M    0     0  72.3M      0 --:--:-- --:--:-- --:--:-- 72.3M


# 2. Configure the Environment Variables

In [6]:
%%bash

source ~/.bash_profile

export AWS_REGION=$(aws configure get region)
echo "export AWS_REGION=${AWS_REGION}" | tee -a ~/.bash_profile

export AWS_CLUSTER_NAME=cluster
echo "export AWS_CLUSTER_NAME=${AWS_CLUSTER_NAME}" | tee -a ~/.bash_profile

echo "Completed"

export AWS_REGION=us-west-2
export AWS_CLUSTER_NAME=cluster
Completed


# 3. Create the EKS Cluster

_Note:  Creating a cluster may take about 15 minutes.  Please be patient._

In [7]:
%%bash

source ~/.bash_profile

cat << EOF > cluster.yaml
---
apiVersion: eksctl.io/v1alpha5
kind: ClusterConfig

metadata:
  name: ${AWS_CLUSTER_NAME}
  region: ${AWS_REGION}

cloudWatch:
  clusterLogging:
    enableTypes: ["*"]

managedNodeGroups:
- name: cpu-nodes
  instanceType: c5.xlarge
  volumeSize: 100
  desiredCapacity: 5
  iam:
    withAddonPolicies:
      albIngress: true

#secretsEncryption:
#  keyARN: ${MASTER_ARN}
EOF

In [8]:
!cat cluster.yaml

---
apiVersion: eksctl.io/v1alpha5
kind: ClusterConfig

metadata:
  name: cluster
  region: us-west-2

cloudWatch:
  clusterLogging:
    enableTypes: ["*"]

managedNodeGroups:
- name: cpu-nodes
  instanceType: c5.xlarge
  volumeSize: 100
  desiredCapacity: 5
  iam:
    withAddonPolicies:
      albIngress: true

#secretsEncryption:
#  keyARN: 


In [9]:
%%bash

source ~/.bash_profile

eksctl create cluster -f ./cluster.yaml

echo "Completed"

[ℹ]  eksctl version 0.17.0
[ℹ]  using region us-west-2
[ℹ]  setting availability zones to [us-west-2d us-west-2a us-west-2b]
[ℹ]  subnets for us-west-2d - public:192.168.0.0/19 private:192.168.96.0/19
[ℹ]  subnets for us-west-2a - public:192.168.32.0/19 private:192.168.128.0/19
[ℹ]  subnets for us-west-2b - public:192.168.64.0/19 private:192.168.160.0/19
[ℹ]  using Kubernetes version 1.15
[ℹ]  creating EKS cluster "cluster" in "us-west-2" region with managed nodes
[ℹ]  1 nodegroup (cpu-nodes) was included (based on the include/exclude rules)
[ℹ]  will create a CloudFormation stack for cluster itself and 0 nodegroup stack(s)
[ℹ]  will create a CloudFormation stack for cluster itself and 1 managed nodegroup stack(s)
[ℹ]  if you encounter any issues, check CloudFormation console or try 'eksctl utils describe-stacks --region=us-west-2 --cluster=cluster'
[ℹ]  Kubernetes API endpoint access will use default of {publicAccess=true, privateAccess=false} for cluster "cluster" in "us-west-2"
[ℹ] 

In [10]:
%%bash

source ~/.bash_profile

eksctl utils describe-stacks --region=${AWS_REGION} --cluster=${AWS_CLUSTER_NAME}

[ℹ]  eksctl version 0.17.0
[ℹ]  using region us-west-2
[ℹ]  stack/eksctl-cluster-nodegroup-cpu-nodes = {
  Capabilities: ["CAPABILITY_IAM"],
  CreationTime: 2020-04-25 20:25:55.219 +0000 UTC,
  Description: "EKS Managed Nodes (SSH access: false) [created by eksctl]",
  DisableRollback: false,
  DriftInformation: {
    StackDriftStatus: "NOT_CHECKED"
  },
  EnableTerminationProtection: false,
  RollbackConfiguration: {

  },
  StackId: "arn:aws:cloudformation:us-west-2:086401037028:stack/eksctl-cluster-nodegroup-cpu-nodes/f7134050-8732-11ea-a34a-06e35a67ba22",
  StackName: "eksctl-cluster-nodegroup-cpu-nodes",
  StackStatus: "CREATE_COMPLETE",
  Tags: [
    {
      Key: "alpha.eksctl.io/cluster-name",
      Value: "cluster"
    },
    {
      Key: "alpha.eksctl.io/nodegroup-name",
      Value: "cpu-nodes"
    },
    {
      Key: "eksctl.cluster.k8s.io/v1alpha1/cluster-name",
      Value: "cluster"
    },
    {
      Key: "alpha.eksctl.io/nodegroup-type",
      Value: "managed"
    },
  

# --- Please wait until the EKS cluster is succesfully deployed before you continue! ---

![](img/verify_eks.png)

# 4. Associate IAM Policies with EKS Worker Nodes


In [11]:
%%bash

### Source the environment
source ~/.bash_profile

### Create more environment variables
export STACK_NAME=$(eksctl get nodegroup --cluster ${AWS_CLUSTER_NAME} -o json | jq -r '.[].StackName')
echo "export STACK_NAME=${STACK_NAME}" | tee -a ~/.bash_profile

export INSTANCE_ROLE_NAME=$(aws iam list-roles \
    | jq -r ".Roles[] \
    | select(.RoleName \
    | startswith(\"eksctl-$AWS_CLUSTER_NAME\") and contains(\"NodeInstanceRole\")) \
    .RoleName")
#export INSTANCE_ROLE_NAME=$(aws cloudformation describe-stacks --stack-name $STACK_NAME --output text --query "Stacks[0].Outputs[1].OutputValue" | sed -e 's/.*\///g')
echo "export INSTANCE_ROLE_NAME=${INSTANCE_ROLE_NAME}" | tee -a ~/.bash_profile

#export INSTANCE_PROFILE_ARN=$(aws cloudformation describe-stacks --stack-name $STACK_NAME | jq -r '.Stacks[].Outputs[] | select(.OutputKey=="InstanceProfileARN") | .OutputValue')
export INSTANCE_PROFILE_ARN=$(aws iam list-roles \
    | jq -r ".Roles[] \
    | select(.RoleName \
    | startswith(\"eksctl-$AWS_CLUSTER_NAME\") and contains(\"NodeInstanceRole\")) \
    .Arn")
echo "export INSTANCE_PROFILE_ARN=${INSTANCE_PROFILE_ARN}" | tee -a ~/.bash_profile

#### Allow Access from/to the Elastic Container Registry (ECR)
# This allows our cluster worker nodes to load custom Docker images (ie. models) from ECR.  We will load these custom Docker images in a later section.
aws iam attach-role-policy --role-name $INSTANCE_ROLE_NAME --policy-arn arn:aws:iam::aws:policy/AmazonEC2ContainerRegistryFullAccess

### Associated IAM and OIDC
# To use IAM roles for service accounts in your cluster, you must create an OIDC identity provider in the IAM console.  See https://docs.aws.amazon.com/eks/latest/userguide/enable-iam-roles-for-service-accounts.html for more info.
eksctl utils associate-iam-oidc-provider --cluster ${AWS_CLUSTER_NAME} --approve
aws eks describe-cluster --name ${AWS_CLUSTER_NAME} --region ${AWS_REGION} --query "cluster.identity.oidc.issuer" --output text

echo "Completed"

export STACK_NAME=eksctl-cluster-nodegroup-cpu-nodes
export INSTANCE_ROLE_NAME=eksctl-cluster-nodegroup-cpu-node-NodeInstanceRole-O5SN4AZLE4GZ
export INSTANCE_PROFILE_ARN=arn:aws:iam::086401037028:role/eksctl-cluster-nodegroup-cpu-node-NodeInstanceRole-O5SN4AZLE4GZ
[ℹ]  eksctl version 0.17.0
[ℹ]  using region us-west-2
[ℹ]  will create IAM Open ID Connect provider for cluster "cluster" in "us-west-2"
[✔]  created IAM Open ID Connect provider for cluster "cluster" in "us-west-2"
https://oidc.eks.us-west-2.amazonaws.com/id/2EE1E69E12E04AEA35251A8390D88EBC
Completed


# 5. Update `~/.kube/config` with our new EKS cluster

In [12]:
%%bash

source ~/.bash_profile

aws eks --region ${AWS_REGION} update-kubeconfig --name ${AWS_CLUSTER_NAME} 

Added new context arn:aws:eks:us-west-2:086401037028:cluster/cluster to /home/ec2-user/.kube/config


# 6. Verify Your EKS Cluster


In [13]:
%%bash

kubectl get ns


NAME              STATUS   AGE
default           Active   6m41s
kube-node-lease   Active   6m43s
kube-public       Active   6m43s
kube-system       Active   6m43s


In [14]:
%%bash

kubectl get nodes


NAME                                           STATUS   ROLES    AGE   VERSION
ip-192-168-25-103.us-west-2.compute.internal   Ready    <none>   95s   v1.15.10-eks-bac369
ip-192-168-45-7.us-west-2.compute.internal     Ready    <none>   95s   v1.15.10-eks-bac369
ip-192-168-62-182.us-west-2.compute.internal   Ready    <none>   96s   v1.15.10-eks-bac369
ip-192-168-68-94.us-west-2.compute.internal    Ready    <none>   89s   v1.15.10-eks-bac369
ip-192-168-75-136.us-west-2.compute.internal   Ready    <none>   94s   v1.15.10-eks-bac369
